In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('D:\ARTIFICIAL_INTELLIGENCE\KỲ 5\Deep_Learning_(DPL302m)\Project'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

D:\ARTIFICIAL_INTELLIGENCE\KỲ 5\Deep_Learning_(DPL302m)\Project\0.27
D:\ARTIFICIAL_INTELLIGENCE\KỲ 5\Deep_Learning_(DPL302m)\Project\75-butterfly-types-image-detection-vit.ipynb
D:\ARTIFICIAL_INTELLIGENCE\KỲ 5\Deep_Learning_(DPL302m)\Project\butterfly-image-classification-mobilenetv2.ipynb
D:\ARTIFICIAL_INTELLIGENCE\KỲ 5\Deep_Learning_(DPL302m)\Project\test.zip
D:\ARTIFICIAL_INTELLIGENCE\KỲ 5\Deep_Learning_(DPL302m)\Project\Testing_set.csv
D:\ARTIFICIAL_INTELLIGENCE\KỲ 5\Deep_Learning_(DPL302m)\Project\train.zip
D:\ARTIFICIAL_INTELLIGENCE\KỲ 5\Deep_Learning_(DPL302m)\Project\Training_set.csv
D:\ARTIFICIAL_INTELLIGENCE\KỲ 5\Deep_Learning_(DPL302m)\Project\test\Image_1.jpg
D:\ARTIFICIAL_INTELLIGENCE\KỲ 5\Deep_Learning_(DPL302m)\Project\test\Image_10.jpg
D:\ARTIFICIAL_INTELLIGENCE\KỲ 5\Deep_Learning_(DPL302m)\Project\test\Image_100.jpg
D:\ARTIFICIAL_INTELLIGENCE\KỲ 5\Deep_Learning_(DPL302m)\Project\test\Image_1000.jpg
D:\ARTIFICIAL_INTELLIGENCE\KỲ 5\Deep_Learning_(DPL302m)\Project\test\Im

In [2]:
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Đường dẫn tới thư mục chứa dữ liệu huấn luyện và kiểm định
train_dir = 'D:/ARTIFICIAL_INTELLIGENCE/KỲ 5/Deep_Learning_(DPL302m)/Project/train'
test_dir = 'D:/ARTIFICIAL_INTELLIGENCE/KỲ 5/Deep_Learning_(DPL302m)/Project/test'

# Đường dẫn tới các tệp CSV chứa thông tin về các tệp hình ảnh
train_csv = 'Training_set.csv'
test_csv = 'Testing_set.csv'

# Đọc dữ liệu từ các tệp CSV vào DataFrame
train_df = pd.read_csv(train_csv)
test_df = pd.read_csv(test_csv)

# Khởi tạo ImageDataGenerator cho dữ liệu huấn luyện và kiểm định
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)
test_datagen = ImageDataGenerator(rescale=1./255)

# Tạo generators cho dữ liệu huấn luyện và kiểm định
train_generator = train_datagen.flow_from_dataframe(
    train_df,
    directory=train_dir,
    x_col='filename',
    y_col='label',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

validation_generator = train_datagen.flow_from_dataframe(
    train_df,
    directory=train_dir,
    x_col='filename',
    y_col='label',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

test_generator = test_datagen.flow_from_dataframe(
    test_df,
    directory=test_dir,
    x_col='ID',
    y_col=None,
    target_size=(224, 224),
    batch_size=32,
    class_mode=None,
    shuffle=False
)


Found 4000 validated image filenames belonging to 75 classes.
Found 1000 validated image filenames belonging to 75 classes.
Found 1499 validated image filenames.


In [3]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
# weights_url = 'https://storage.googleapis.com/tensorflow/keras-applications/mobilenet_v2/mobilenet_v2_weights_tf_dim_ordering_tf_kernels_1.0_224_no_top.h5'
# weights_path = tf.keras.utils.get_file('mobilenet_v2_weights.h5', weights_url)
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
predictions = Dense(75, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

for layer in base_model.layers:
    layer.trainable = False

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [4]:
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.n // train_generator.batch_size,
    epochs=10,
    validation_data=validation_generator,
    validation_steps=validation_generator.n // validation_generator.batch_size
)

Epoch 1/10


C:\Users\Acer\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


  9/125 ━━━━━━━━━━━━━━━━━━━━ 1:27 757ms/step - accuracy: 0.0542 - loss: 4.4513

In [ ]:
val_loss, val_accuracy = model.evaluate(validation_generator)
print(f'Validation accuracy: {val_accuracy * 100:.2f}%')

In [ ]:
for layer in base_model.layers[-20:]:
    layer.trainable = True

from tensorflow.keras.optimizers import Adam
model.compile(optimizer=Adam(learning_rate=1e-5),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

history_fine = model.fit(
    train_generator,
    steps_per_epoch=train_generator.n // train_generator.batch_size,
    epochs=10,
    validation_data=validation_generator,
    validation_steps=validation_generator.n // validation_generator.batch_size
)

In [ ]:
val_loss, val_accuracy = model.evaluate(validation_generator)
print(f'Validation accuracy: {val_accuracy * 100:.2f}%')

In [ ]:
predictions = model.predict(test_generator)

predicted_classes = predictions.argmax(axis=1)
class_labels = list(train_generator.class_indices.keys())
predicted_labels = [class_labels[i] for i in predicted_classes]

test_df['label'] = predicted_labels
test_df.to_csv('predictions.csv', index=False)

In [ ]:
predictions = pd.read_csv('/kaggle/working/predictions.csv')
predictions.head(20)